# Руководство по кодированию категориальных значений в Python

<a href="https://t.me/init_python"><img src="https://dfedorov.spb.ru/pandas/logo-telegram.png" width="35" height="35" alt="telegram" align="left"></a>

<a href="https://colab.research.google.com/github/dm-fedorov/pandas_basic/blob/master/быстрое%20введение%20в%20pandas/Руководство%20по%20кодированию%20категориальных%20значений%20в%20Python.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory" target="_blank"></a>

# Введение

Часто наборы данных содержат категориальные переменные.

> дополнительно см. статью [Использование типа данных категории в pandas](https://dfedorov.spb.ru/pandas/%D0%98%D1%81%D0%BF%D0%BE%D0%BB%D1%8C%D0%B7%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5%20%D1%82%D0%B8%D0%BF%D0%B0%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85%20%D0%BA%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D0%B8%20%D0%B2%20pandas.html). 

Эти переменные обычно хранятся в виде текстовых значений, которые представляют различные характеристики. Некоторые примеры включают цвет ("Красный", "Желтый", "Синий"), размер ("Маленький", "Средний", "Большой") или географические обозначения ("Штат" или "Страна"). 

Многие алгоритмы машинного обучения поддерживают категориальные значения без дополнительных манипуляций, но есть множество алгоритмов, которые этого не делают. Следовательно, перед аналитиком стоит задача выяснить, как *преобразовать эти текстовые атрибуты в числовые значения* для дальнейшей обработки.

> Оригинал статьи Криса по [ссылке](https://pbpython.com/categorical-encoding.html)

Как и во многих других аспектах здесь нет однозначного ответа. Каждый подход имеет свои плюсы/минусы и может повлиять на результат анализа. К счастью, инструменты *Python*, такие как *pandas* и *scikit-learn*, предоставляют несколько методик. Эта статья является обзором популярных (и более сложных) подходов в надежде, что это поможет другим применить рассмотренные методы к своим задачам.

# Набор данных

Для этой статьи мне удалось найти хороший набор данных в [репозитории машинного обучения UCI](https://archive.ics.uci.edu/ml/index.php). Этот [автомобильный набор данных](https://archive.ics.uci.edu/ml/datasets/automobile) включает хорошее сочетание категориальных, а также непрерывных значений и служит полезным примером. Поскольку понимание предметной области является важным аспектом при принятии решения о том, как кодировать различные категориальные значения, этот набор данных является хорошим примером.

Прежде чем мы начнем кодировать значения, нам нужно произвести небольшую очистку. 

К счастью, в *pandas* это делается просто:

In [ ]:
import pandas as pd
import numpy as np

# Определите заголовки, так как данные их не содержат
headers = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration",
           "num_doors", "body_style", "drive_wheels", "engine_location",
           "wheel_base", "length", "width", "height", "curb_weight",
           "engine_type", "num_cylinders", "engine_size", "fuel_system",
           "bore", "stroke", "compression_ratio", "horsepower", "peak_rpm",
           "city_mpg", "highway_mpg", "price"]

In [ ]:
# Прочтите CSV-файл и преобразуйте "?" в NaN
df = pd.read_csv("https://github.com/dm-fedorov/pandas_basic/blob/master/%D0%B1%D1%8B%D1%81%D1%82%D1%80%D0%BE%D0%B5%20%D0%B2%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D0%B5%20%D0%B2%20pandas/data/imports-85.data?raw=True",
                  header=None, names=headers, na_values="?" )
df.head()

Посмотрим, какие типы данных у нас есть:

In [ ]:
df.dtypes

Поскольку в этой статье мы сосредоточимся только на кодировании категориальных переменных, мы собираемся включить в наш фрейм данных только столбцы типа `object`. 

> дополнительно см. статью [Обзор типов данных Pandas]((https://dfedorov.spb.ru/pandas/%D0%9E%D0%B1%D0%B7%D0%BE%D1%80%20%D1%82%D0%B8%D0%BF%D0%BE%D0%B2%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85%20pandas.html))

В *pandas* есть полезная функция [`select_dtypes`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.select_dtypes.html), которую можно использовать для создания нового фрейма данных, содержащего только столбцы типа `object`:

In [ ]:
obj_df = df.select_dtypes(include=['object']).copy()

In [ ]:
obj_df.head()

Прежде чем идти дальше, в данных есть пара нулевых значений, которые необходимо очистить:

In [ ]:
obj_df[obj_df.isnull().any(axis=1)]

Для простоты заполните значение числом `four` (так как это наиболее распространенное значение):

In [ ]:
obj_df["num_doors"].value_counts()

In [ ]:
obj_df = obj_df.fillna({"num_doors": "four"})

In [ ]:
obj_df[obj_df.isnull().any(axis=1)]

In [ ]:
obj_df.head()

Теперь, когда данные не имеют нулевых значений, можем рассмотреть варианты кодирования категориальных значений.

## Подход №1 - Найти и заменить

Прежде чем мы перейдем к более "стандартным" подходам кодирования категориальных данных, этот набор данных включает один потенциальный подход, который я называю *"найти и заменить"*.

Есть два столбца данных, значения которых представляют собой слова, используемые для представления чисел. В частности, количество цилиндров в двигателе (`num_cylinders`) и количество дверей в машине (`num_doors`). *Pandas* позволяет нам напрямую заменять текстовые значения их числовыми эквивалентами, используя `replace`.

Мы уже видели, что данные `num_doors` включают только `2` или `4` двери. Количество цилиндров включает всего `7` значений, которые легко переводятся в действительные числа:

In [ ]:
obj_df["num_cylinders"].value_counts()

Если вы просмотрите [документацию по `replace`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html), то увидите, что это мощная функция с множеством параметров. 

Для наших целей мы создадим словарь сопоставления (*mapping*), содержащий столбец для обработки (ключ словаря), а также словарь значений для замены.

Вот полный словарь для очистки столбцов `num_doors` и `num_cylinders`:

In [ ]:
cleanup_nums = {"num_doors":     {"four": 4, "two": 2},
                "num_cylinders": {"four": 4, "six": 6, "five": 5, "eight": 8,
                                  "two": 2, "twelve": 12, "three":3 }}

Чтобы преобразовать столбцы в числа с помощью `replace`:

In [ ]:
obj_df = obj_df.replace(cleanup_nums)

In [ ]:
obj_df.head()

Хорошим преимуществом этого подхода является то, что *pandas* "знает" типы значений в столбцах, поэтому теперь объект имеет тип `int64`:

In [ ]:
obj_df.dtypes

Хотя данный подход может работать только в определенных случаях, это очень полезная демонстрация того, как преобразовать текстовые значения в числовые, когда есть "легкая" интерпретация данных человеком. Представленная концепция также полезна для более общей очистки данных.

## Подход № 2 - Кодирование метки

Другой подход к кодированию категориальных значений заключается в использовании метода, называемого кодированием меток (`label encoding`). 

Кодирование метки - это простое преобразование каждого значения в столбце в число. Например, столбец `body_style` содержит `5` разных значений. Мы могли бы закодировать это так:

- `кабриолет (convertible) -> 0`
- `"жесткий верх" (hardtop) -> 1`
- `хэтчбек (hatchback) -> 2`
- `седан (sedan) -> 3`
- `"вэгон" (wagon) -> 4`

Прием, который вы можете использовать в *pandas*, - это преобразовать столбец в категорию, а затем использовать эти значения категории для кодирования метки:

In [ ]:
obj_df["body_style"].value_counts()

In [ ]:
obj_df["body_style"] = obj_df["body_style"].astype('category')

In [ ]:
obj_df.dtypes

Затем вы можете назначить закодированную переменную новому столбцу с помощью метода доступа (*accessor*) `cat.codes`:

In [ ]:
obj_df["body_style_cat"] = obj_df["body_style"].cat.codes
obj_df.head()

In [ ]:
obj_df.dtypes

Приятным аспектом этого подхода является то, что вы получаете [преимущества категорий *pandas*](https://dfedorov.spb.ru/pandas/%D0%98%D1%81%D0%BF%D0%BE%D0%BB%D1%8C%D0%B7%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5%20%D1%82%D0%B8%D0%BF%D0%B0%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85%20%D0%BA%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D0%B8%20%D0%B2%20pandas.html) (компактный размер данных, возможность упорядочивания, поддержка построения графиков), но их можно легко преобразовать в числовые значения для дальнейшего анализа.

## Подход № 3 - Унитарное кодирование (One Hot Encoding)

Кодирование меток имеет тот недостаток, что числовые значения могут быть "неверно интерпретированы" алгоритмами. Например, значение `0` очевидно меньше значения `4`, но действительно ли это соответствует набору данных в реальной жизни? Имеет ли универсал в `4` раза больший вес, чем у кабриолета? В этом примере я так не думаю.

Общий альтернативный подход называется *унитарным кодированием* (*One Hot Encoding*). Основная стратегия состоит в том, чтобы преобразовать значение каждой категории в новый столбец и присвоить столбцу значение `1` или `0` (*Истина / Ложь*). Это дает преимущество в том, что значение не взвешивается неправильно, но имеет обратную сторону добавления дополнительных столбцов в набор данных.

*Pandas* поддерживает эту возможность с помощью [`get_dummies`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html). Эта функция названа так, потому что она создает фиктивные (*dummy*) / индикаторные переменные (`1` или `0`).

Надеюсь, простой пример прояснит это. Мы можем посмотреть на столбец `drive_wheels`, где у нас есть значения `4wd`, `fwd` или `rwd`. 

Используя `get_dummies`, мы можем преобразовать их в три столбца с `1` или `0`, соответствующими правильному значению:

In [ ]:
pd.get_dummies(obj_df, columns=["drive_wheels"]).head()

Новый набор данных содержит три новых столбца:

- `drive_wheels_4wd`
- `drive_wheels_rwd`
- `drive_wheels_fwd`

Эта мощная функция, потому что вы можете передать столько столбцов категорий, сколько захотите, и выбрать, как обозначить столбцы с помощью префикса. 

Правильное присвоение имен немного упростит дальнейший анализ:

In [ ]:
pd.get_dummies(obj_df, columns=["body_style", "drive_wheels"], prefix=["body", "drive"]).head()

Другая концепция, о которой следует помнить, заключается в том, что `get_dummies` возвращает полный фрейм данных (*dataframe*), поэтому вам нужно будет отфильтровать объекты с помощью `select_dtypes` перед проведением итогового анализа.

> Унитарное кодирование очень полезно, но оно может увеличить количество столбцов, если у вас много уникальных значений в столбце. 

## Подход №4 - Пользовательское двоичное кодирование

В зависимости от набора данных вы можете использовать некоторую комбинацию кодирования меток и унитарного кодирования для создания двоичного столбца, который соответствует вашим потребностям.

В этом конкретном наборе данных есть столбец с именем `engine_type`, который содержит несколько разных значений:

In [ ]:
obj_df["engine_type"].value_counts()

Возможно, нас волнует, оснащен ли двигатель верхним распредвалом (*Overhead Cam, OHC*) или нет. 

Другими словами, разные версии *OHC* одинаковы для этого анализа. Если это так, то мы могли бы использовать метод доступа (*accessor*) `str` и `np.where` (функциональная замена условия) для создания нового столбца, который указывает, есть ли в автомобиле двигатель OHC:

In [ ]:
obj_df["OHC_Code"] = np.where(obj_df["engine_type"].str.contains("ohc"), 1, 0)

Это удобная функция, но иногда я забываю о синтаксисе, поэтому вот график, показывающий, что мы делаем:

![](https://github.com/dm-fedorov/pandas_basic/blob/master/pic/np-where-2.png?raw=True)

Результирующий фрейм данных выглядит следующим образом (показывает только подмножество столбцов):

In [ ]:
obj_df[["make", "engine_type", "OHC_Code"]].head()

Это подход подчеркивает, насколько важно знание предметной области для эффективного решения проблемы.

## scikit-Learn

В дополнение к подходу *pandas*, *scikit-learn* предоставляет [аналогичную функциональность](https://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features). Лично я считаю, что использование *pandas* проще для понимания, но подход *scikit* является оптимальным, когда вы пытаетесь построить прогнозную модель.

Например, если мы хотим выполнить кодирование меток для марки автомобиля (*make*), нам нужно создать экземпляр объекта `OrdinalEncoder` и произвести `fit_transform` данных:

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ord_enc = OrdinalEncoder()

In [ ]:
obj_df["make_code"] = ord_enc.fit_transform(obj_df[["make"]])

In [ ]:
obj_df[["make", "make_code"]].head(11)

*scikit-learn* также поддерживает двоичное кодирование с помощью `OneHotEncoder`. 

Мы используем тот же процесс, что и выше, для преобразования данных, но процесс создания фрейма данных (*DataFrame*) добавляет пару дополнительных шагов.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
oe_style = OneHotEncoder()

In [ ]:
oe_results = oe_style.fit_transform(obj_df[["body_style"]])

Результатом является массив, который необходимо преобразовать во фрейм данных:

In [ ]:
oe_results.toarray()

In [ ]:
pd.DataFrame(oe_results.toarray(), columns=oe_style.categories_).head()

Следующим шагом будет присоединение этих данных обратно к исходному фрейму. 

Вот пример:

In [ ]:
obj_df = obj_df.join(pd.DataFrame(oe_results.toarray(), columns=oe_style.categories_))
obj_df.head()

Ключевым моментом является то, что вам нужно использовать `toarray()` для преобразования результатов в формат, который можно преобразовать во фрейм данных.

## Продвинутые подходы

Есть еще более продвинутые алгоритмы категориального кодирования. У меня нет опыта работы с ними, но, чтобы завершить это руководство, я захотел их включить. В [этой статье](http://www.willmcginnis.com/2015/11/29/beyond-one-hot-an-exploration-of-categorical-variables/) содержится дополнительная техническая информация. 

Другой приятный аспект заключается в том, что автор статьи создал пакет для *scikit-learn* под названием [`category_encoders`](https://github.com/scikit-learn-contrib/category_encoders), который реализует многие из этих подходов. Это очень хороший инструмент, позволяющий взглянуть на проблему с другой точки зрения.

In [ ]:
#!pip3 install category_encoders

В первом примере мы попробуем выполнить [кодирование обратной разницы](https://contrib.scikit-learn.org/category_encoders/backward_difference.html) (*Backward Difference encoding*).

Сначала мы получаем чистый фрейм данных и настраиваем `BackwardDifferenceEncoder`:

In [ ]:
import category_encoders as ce

In [ ]:
# Получите новый чистый фрейм данных
obj_df = df.select_dtypes(include=['object']).copy()

In [ ]:
# Укажите столбцы для кодирования, затем выполните fit и transform
encoder = ce.BackwardDifferenceEncoder(cols=["engine_type"])

In [ ]:
encoder.fit(obj_df, verbose=1)

# https://stackoverflow.com/questions/63589556/getting-is-categorical-is-deprecated-error-while-using-jamessteinencoder

In [ ]:
encoder.fit_transform(obj_df, verbose=1).iloc[:,8:14].head()

Интересно то, что результат не соответствует стандартным единицам и нулям, которые мы видели в предыдущих примерах кодирования. 

Если мы попробуем [полиномиальное кодирование](https://contrib.scikit-learn.org/category_encoders/polynomial.html) (*polynomial encoding*), то получим другое распределение значений, используемых для кодирования столбцов:

In [ ]:
encoder = ce.PolynomialEncoder(cols=["engine_type"])

In [ ]:
encoder.fit_transform(obj_df, verbose=1).iloc[:,8:14].head()

В этот пакет включено несколько различных алгоритмов, и лучший способ изучить их - попробовать их и посмотреть, поможет ли это повысить точность вашего анализа. 

## Конвейеры scikit-learn

Цель этого раздела показать, как интегрировать особенности функций кодирования *scikit-learn* в простой конвейер (*pipeline*) построения модели.

Как упоминалось выше, категориальные кодировщики *scikit-learn* позволяют включать преобразование в ваши конвейеры, что позволяет упростить процесс построения модели и избежать некоторых ошибок. Я рекомендую [это видео](https://www.dataschool.io/encoding-categorical-features-in-python/) в качестве хорошего вступления. Оно послужило основой для изложенного ниже подхода.

Вот очень быстрый пример того, как включить `OneHotEncoder` и `OrdinalEncoder` в конвейер и использовать `cross_val_score` для анализа результатов:

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

# для целей этого анализа используйте только небольшой набор признаков
feature_cols = [
    'fuel_type', 'make', 'aspiration', 'highway_mpg', 
    'city_mpg', 'curb_weight', 'drive_wheels'
]

# Удалите пустые строки с ценами
df_ml = df.dropna(subset=['price'])

X = df_ml[feature_cols]
y = df_ml['price']

Теперь, когда у нас есть данные, давайте создадим преобразователь (transformer) столбцов:

In [ ]:
column_trans = make_column_transformer((OneHotEncoder(handle_unknown='ignore'),
                                        ['fuel_type', 'make', 'drive_wheels']),
                                      (OrdinalEncoder(), ['aspiration']),
                                      remainder='passthrough')

В этом примере показано, как применять разные типы кодировщиков для определенных столбцов. 

Используем аргумент `restder='passthrough'` для передачи всех числовых значений через конвейер без каких-либо изменений.

Для модели мы используем простую линейную регрессию, а затем создаем конвейер:

In [ ]:
linreg = LinearRegression()
pipe = make_pipeline(column_trans, linreg)

Выполните перекрестную проверку (*cross validation*) `10` раз, используя *отрицательную среднюю абсолютную ошибку* (`neg_mean_absolute_error`) в качестве функции оценки. 

Наконец, возьмите среднее из `10` значений, чтобы увидеть величину ошибки:

In [ ]:
cross_val_score(pipe, X, y, cv=10, scoring='neg_mean_absolute_error').mean().round(2)

Очевидно, что здесь можно провести гораздо больше анализа, но это сделано для того, чтобы проиллюстрировать, как использовать функции *scikit-learn* в более реалистичном конвейере анализа.

# Заключение

Кодирование категориальных переменных - важный шаг в процессе анализа данных. Поскольку существует несколько подходов к кодированию переменных, важно понимать различные варианты и способы их реализации в ваших собственных наборах данных. В экосистеме науки о данных *Python* есть много полезных подходов к решению этих проблем. Я призываю вас помнить об этих идеях в следующий раз, когда вы обнаружите, что анализируете категориальные переменные. 